In [32]:
import requests
import pandas as pd
import numpy as np

In [38]:
api_key = "TJI6M4BFN9J1591S49US8UZ7MSZFX5JK8Y"
address = "0xdbf5e9c5206d0db70a90108bf936da60221dc080"
contract_address = "0xa62cc35625B0C8dc1fAEA39d33625Bb4C15bD71C" ## STMX

#drawl Token Transfer Events' by Address from etherscan api 
#https://docs.etherscan.io/api-endpoints/accounts#get-a-list-of-erc20-token-transfer-events-by-address
url = f"https://api.etherscan.io/api?module=account&action=tokentx&contractaddress={contract_address}&address={address}&page=1&offset=100&sort=desc&apikey={api_key}"
response = requests.get(url)
data = response.json()
transactions = data["result"]

#select relevant columns 
df_token_transfer = pd.DataFrame(transactions)
df_token_transfer = df_token_transfer[["timeStamp", "hash", "from","to","tokenName","tokenSymbol","value","tokenDecimal"]]

#convert datatype
df_token_transfer["tokenDecimal"] = np.int64(df_token_transfer["tokenDecimal"])
df_token_transfer["value_after"] = df_token_transfer["value"].astype(float) / pow(10,df["tokenDecimal"])
df_token_transfer["timeStamp"] = pd.to_datetime(df_token_transfer["timeStamp"], unit="s")

df_token_transfer.head()

,timeStamp,hash,from,to,tokenName,tokenSymbol,value,tokenDecimal,value_after
0,2023-03-25 03:03:59,0x9df6e6164ede841561e0bb4af35add8fe55a759b331e...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,59600000000000000000000000,18,59600000.0
1,2023-03-25 02:50:35,0xa2c86f0e206f64ed8c490726c3cb31ed5cf9fd6327c1...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,2000000000000000000000,18,2000.0
2,2023-03-24 21:02:47,0x605568a699c5f168303babb88c7d9f1e06b7f87e88b5...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,5000000000000000000000000,18,5000000.0
3,2023-03-23 21:23:47,0xf872e338fcee84d8788c73368553bf0e4ce774062637...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,34739153000000000000000000,18,34739153.0
4,2023-03-23 17:32:59,0x8bc4d0ece9959010897dd40848eaf4ae578e8a0d09ad...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,34274402000000000000000000,18,34274402.0


In [53]:
from_day = int(round(df_token_transfer["timeStamp"].iloc[-1].timestamp()))
to_day = int(round(df_token_transfer["timeStamp"].iloc[1].timestamp()))
currency = 'usd'

# Get historical market data include price, market cap, and 24h volume within a range of timestamp
# https://www.coingecko.com/en/api/documentation
url1 = f"https://api.coingecko.com/api/v3/coins/1/contract/{contract_address}/market_chart/range?vs_currency={currency}&from={from_day}&to={to_day}"
response = requests.get(url1)
data = response.json()

df_token = pd.DataFrame(data)
df_token[["timestamp", "price_value"]] = df_token["prices"].apply(lambda x: pd.Series([x[0], x[1]]))
df_token["timestamp"] = np.int64(df_token["timestamp"])

# Sorting by date
df_token.sort_values(by = ["timestamp"], ascending = False, inplace = True)
df_token["timestamp"] = pd.to_datetime(df_token["timestamp"], unit="ms")

df_token_price = df_token[["timestamp", "price_value"]]
df_token_price

,timestamp,price_value
75,2023-03-25 02:01:15.703,0.005666
74,2023-03-25 01:01:50.304,0.005670
73,2023-03-25 00:01:45.055,0.005639
72,2023-03-24 23:01:24.317,0.005621
71,2023-03-24 22:01:56.971,0.005601
...,...,...
4,2023-03-22 03:02:05.964,0.005913
3,2023-03-22 02:02:07.066,0.005909
2,2023-03-22 01:01:59.565,0.005920
1,2023-03-22 00:02:02.126,0.006045


In [54]:
# calculate time difference
time_diff = []
for timestamp in df_token_transfer['timeStamp']:
    time_diff.append((df_token_price['timestamp'] - timestamp).abs().idxmin())

# get corresponding price from df_token_price
prices = df_token_price.loc[time_diff, 'price_value'].values

# add prices as new column in df_token_transfer
df_token_transfer['Prices'] = prices
df_token_transfer.head()

,timeStamp,hash,from,to,tokenName,tokenSymbol,value,tokenDecimal,value_after,Prices
0,2023-03-25 03:03:59,0x9df6e6164ede841561e0bb4af35add8fe55a759b331e...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,59600000000000000000000000,18,59600000.0,0.005666
1,2023-03-25 02:50:35,0xa2c86f0e206f64ed8c490726c3cb31ed5cf9fd6327c1...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,2000000000000000000000,18,2000.0,0.005666
2,2023-03-24 21:02:47,0x605568a699c5f168303babb88c7d9f1e06b7f87e88b5...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,5000000000000000000000000,18,5000000.0,0.005643
3,2023-03-23 21:23:47,0xf872e338fcee84d8788c73368553bf0e4ce774062637...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,34739153000000000000000000,18,34739153.0,0.005780
4,2023-03-23 17:32:59,0x8bc4d0ece9959010897dd40848eaf4ae578e8a0d09ad...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,StormX,STMX,34274402000000000000000000,18,34274402.0,0.005874
